In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler, StandardScaler

import torch
from torch import nn
from torch.autograd import Variable

In [23]:
Xinp = pd.read_csv('feat_test.csv').drop('object_id', axis=1)
Xinp = np.nan_to_num(Xinp.values)
Xinp = StandardScaler().fit_transform(Xinp)
print(Xinp.shape)

(3492890, 68)


In [12]:

batch_size = 10000

batch_num = Xinp.shape[0]//batch_size
Xs = [Xinp[i*batch_size : (i+1)*batch_size] for i in range(0, batch_num)]

assert batch_num == len(Xs)
assert all([s.shape[0] == batch_size for s in Xs])

In [15]:
learning_rate = 0.005
num_epochs = 130

In [16]:
class autoencoder(nn.Module):
    def __init__(self, input_size, latent_size):
        super(autoencoder, self).__init__()

        if input_size <= latent_size + 2:
            raise Exception("input size is not enough bigger than latent size")
        
        self.input_size = input_size
        self.latent_size = latent_size
        self.mid_size = (input_size + latent_size) // 2 

        self.encoder = nn.Sequential(
            nn.Linear(input_size, self.mid_size),
            nn.ReLU(True),
            nn.Linear(self.mid_size, latent_size)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_size, self.mid_size),
            nn.ReLU(True),
            nn.Linear(self.mid_size, input_size),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [17]:
model = autoencoder(input_size=Xs[0].shape[1],
                    latent_size=20
                   ).cuda(2)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,
                             weight_decay=1e-5)

In [18]:

for epoch in range(num_epochs):
    for X in Xs:
        X = Variable(torch.tensor(X.astype(np.float32))).cuda(2)
        # ===================forward=====================
        output = model(X)
        loss = criterion(output, X)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    if epoch % 10 == 0:
        print('epoch [{}/{}], loss:{:.4f}'
              .format(epoch+1, num_epochs, loss.data.item()))

epoch [1/130], loss:0.6569
epoch [11/130], loss:0.1661
epoch [21/130], loss:0.1045
epoch [31/130], loss:0.1198
epoch [41/130], loss:0.0826
epoch [51/130], loss:0.1009
epoch [61/130], loss:0.0722
epoch [71/130], loss:0.0941
epoch [81/130], loss:0.0913
epoch [91/130], loss:0.0689
epoch [101/130], loss:0.0916
epoch [111/130], loss:0.0620
epoch [121/130], loss:0.0667
